In [1]:
!pip install librosa midiutil pydub moviepy mido pyFluidSynth

In [4]:
import fluidsynth
import mido
import librosa
import numpy as np
from midiutil import MIDIFile
from pydub import AudioSegment
from pydub.playback import play
from moviepy.editor import VideoFileClip

# Function to map frequencies to piano notes
def freq_to_midi(freq):
    if freq <= 0:
        return None
    # Convert frequency to MIDI note (A440 is MIDI note 69)
    return int(round(69 + 12 * np.log2(freq / 440.0)))

# Load the audio file and extract pitches
def extract_pitches(audio_file):
    # Load audio
    y, sr = librosa.load(audio_file)

    # Use librosa's pitch detection algorithm
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)

    # Extract the strongest pitches
    extracted_pitches = []
    for t in range(pitches.shape[1]):
        index = magnitudes[:, t].argmax()
        pitch = pitches[index, t]
        if pitch > 0:  # Only consider positive pitches
            extracted_pitches.append(pitch)
    
    return extracted_pitches

# Create a MIDI file from the extracted pitches
def create_midi_from_pitches(pitches, output_file="output.mid"):
    # Create a MIDI track
    midi = MIDIFile(1)  # One track
    midi.addTempo(0, 0, 120)  # Track 0, Time 0, Tempo 120

    time = 0  # Start time
    for pitch in pitches:
        midi_note = freq_to_midi(pitch)
        if midi_note:  # Ignore invalid frequencies
            midi.addNote(0, 0, midi_note, time, 1, 100)  # Channel 0, Pitch, Duration, Volume
            time += 1  # Move to the next time slot

    # Write the MIDI file to disk
    with open(output_file, "wb") as f:
        midi.writeFile(f)

def play_midi(midi_file, soundfont_file="soundfonts/CT2MGM.SF2"):
    """Plays a MIDI file using FluidSynth."""

    # Initialize FluidSynth
    fs = fluidsynth.Synth()
    fs.start()

    # Load the soundfont
    sfid = fs.sfload(soundfont_file)
    fs.program_select(0, sfid, 0, 0)  # Channel 0, bank 0, preset 0

    # Open the MIDI file
    mid = mido.MidiFile(midi_file)

    # Play the MIDI file
    for msg in mid.play():
        if msg.type == 'note_on':
            fs.noteon(msg.channel, msg.note, msg.velocity)
        elif msg.type == 'note_off':
            fs.noteoff(msg.channel, msg.note)

    # Cleanup
    fs.delete()


# Main execution
#video = VideoFileClip("/Users/250003295/Downloads/CareCompanionQ3demo.mp4")
#video.audio.subclip(0, 10).write_audiofile('input_audio.wav')
audio_file = "input_audio.wav"  # Replace with your input file path
pitches = extract_pitches(audio_file)
create_midi_from_pitches(pitches)
play_midi("output.mid")

fluidsynth: warning: Preset 'Melo Tom 2': Some invalid generators were discarded


In [5]:
from midi2audio import FluidSynth

# Initialize FluidSynth with a soundfont (optional)
fs = FluidSynth() 

# Convert MIDI to WAV
fs.midi_to_audio('output.mid', 'output.wav') 

fluidsynth: error: fluid_is_soundfont(): fopen() failed: 'File does not exist.'
Parameter '/Users/250003295/.fluidsynth/default_sound_font.sf2' not a SoundFont or MIDI file or error occurred identifying it.
fluidsynth: error: fluid_sfloader_load(): Failed to open '/opt/homebrew/Cellar/fluid-synth/2.3.6/share/soundfonts/default.sf2': File does not exist.
fluidsynth: error: Unable to open file '/opt/homebrew/Cellar/fluid-synth/2.3.6/share/soundfonts/default.sf2'
fluidsynth: error: Failed to load SoundFont "/opt/homebrew/Cellar/fluid-synth/2.3.6/share/soundfonts/default.sf2"


FluidSynth runtime version 2.3.6
Copyright (C) 2000-2024 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'output.wav'..
